In [ ]:
import pandas as pd 
import numpy as np 


In [ ]:
# train data 
trn = pd.read_csv('../input/google-smartphone-decimeter-challenge/baseline_locations_train.csv')
trn.head()

In [ ]:
# test data
tst = pd.read_csv('../input/google-smartphone-decimeter-challenge/baseline_locations_test.csv')
tst.head()

In [ ]:
trn.shape , tst.shape

In [ ]:
# find distinct values in test and train data set 
for col in trn.columns:
    print('{0} : (trn , tst) : {1}, {2} '.format(col, trn[col].nunique(), trn[col].nunique()))

In [ ]:
## Check min , max and percentile of the dataset
trn.describe()

In [ ]:
tst.describe()

In [ ]:
# change data type of millisSinceGpsEpoch
Epoch_offset = pd.to_datetime('1980-01-06 00:00:00')
#print(Epoch_offset.value)
Epoch_offset_in_ms = int(Epoch_offset.value / 1e6)
print(Epoch_offset_in_ms)


In [ ]:
trn['millisSinceGpsEpoch'] = pd.to_datetime(trn['millisSinceGpsEpoch'] + Epoch_offset_in_ms , unit='ms' )
tst['millisSinceGpsEpoch'] = pd.to_datetime(tst['millisSinceGpsEpoch'] + Epoch_offset_in_ms , unit='ms' )

In [ ]:
trn.head()

In [ ]:
# latdeg and lngdeg
trn_lat_lng = trn[['latDeg','lngDeg']].round(3)
trn_lat_lng['count'] = 1
trn_lat_lng_f  = trn_lat_lng.groupby(['latDeg','lngDeg']).sum().reset_index()
trn_lat_lng_f[trn_lat_lng_f['count'] != 1]

In [ ]:
# latdeg and lngdeg
tst_lat_lng = tst[['latDeg','lngDeg']].round(3)
tst_lat_lng['count'] = 1
tst_lat_lng_f  = tst_lat_lng.groupby(['latDeg','lngDeg']).sum().reset_index()
tst_lat_lng_f[tst_lat_lng_f['count'] != 1]

In [ ]:
import folium 
lat1 = trn.latDeg[0]
lng1 = trn.lngDeg[0]
m = folium.Map(location=[lat1, lng1])
for i in range(1,len(tst_lat_lng_f)):
    folium.Marker(location=[trn.latDeg[i], trn.lngDeg[i]],popup='Default popup Marker1').add_to(m)
m

# Region / Model Phone / Log Analysis

In [ ]:
# Check first log to find data 
#('../input/google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-1/Pixel4/Pixel4_GnssLog.txt');

def ReadGNSSlog(train_test, drive_id, phone_name, log_type):
    '''Read log file . Extract headers and then create data frame under each header'''
    path= '../input/google-smartphone-decimeter-challenge/'+train_test+'/'+drive_id+'/'+phone_name+'/'+phone_name+'_GnssLog.txt'
    #print('Path is {0}'.format(path))
    header = [log_type]
    header_dict = {i: [] for i in header}    
    datas = {i: [] for i in header}
    
    with open(path) as f_open:
        datalines = f_open.read().splitlines()
        
        for line in datalines:
            #print(line)
            is_header = line.startswith('#')
            line = line.strip('#').strip(' ').split(',')
            #print('after change {0}'.format(line))
            if (is_header and line[0] in header):
                header_dict[line[0]] = ['collectionName', 'phoneName']+line[1:]
            elif not is_header and line[0] in header:
               # datas[line[0]].append([drive_id,phone_name ])
                datas[line[0]].append([drive_id,phone_name ]+ line[1:])
        
        results= dict()
        for key, val in datas.items():
            #df_name = print('df_{0}'.format(key))
            #print(df_name)
            #exec(f"df_{key} = pd.DataFrame(datas[key] , columns= header_dict[key])")
            results[key] = pd.DataFrame(val, columns=header_dict[key])
            print(key)
            
    
        f_open.close()
        return results

In [ ]:
trn_GNSS_data = trn[['collectionName', 'phoneName']].groupby(['collectionName', 'phoneName']).sum().reset_index()
trn_GNSS_data.head()

In [ ]:
results_f={}
for row in range(len(trn_GNSS_data)):
    print(trn_GNSS_data.loc[row,'collectionName'] , trn_GNSS_data.loc[row,'phoneName'])
    key = 'results_train_'+trn_GNSS_data.loc[row,'collectionName']+'_'+trn_GNSS_data.loc[row,'phoneName']
    print('.....'+key+'....')
    results_f[key]= ReadGNSSlog('train', trn_GNSS_data.loc[row,'collectionName'], trn_GNSS_data.loc[row,'phoneName'],'Raw')

In [ ]:
first = True
for key in results_f.keys():
    if (first):
        df_train_Raw = results_f[key]["Raw"]
        first = False
    else:
        #print( df_train_Raw.head(5))
        #print(results_f[key]["Raw"].head(5))
        df_train_Raw = pd.concat([df_train_Raw, results_f[key]["Raw"]], ignore_index=True )

df_train_Raw.head()

In [ ]:
# Shape of df_train_Raw
df_train_Raw.shape

In [ ]:
df_train_Raw.info()

EDA for GNSS Logs : Raw Data

In [ ]:
"I am error "
cols = list(df_train_Raw.columns.difference(['collectionName','phoneName']))
df_train_Raw[cols]= df_train_Raw[cols].apply(pd.to_numeric, errors='coerce')
df_train_Raw = df_train_Raw.replace(np.nan, 0)
df_train_Raw.head()

In [ ]:
# Clear intermediate dictionary
results_f.clear()
df_train_Raw = df_train_Raw.replace(np.nan, 0)
df_train_Raw.head()

In [ ]:
# Info of new dataframe
df_train_Raw.info(verbose=True)

In [ ]:
import matplotlib.pyplot as plt
# Calculate Arrival Time 
df_Raw['ArrivalTime'] = df_Raw['TimeNanos'] - df_Raw['FullBiasNanos'] - df_Raw['BiasNanos']
print(df_Raw.ArrivalTime.describe())
df_Raw.ArrivalTime.hist(bins=20)
plt.show()

In [ ]:
df_Raw.info()

In [ ]:
df_Raw['TransmitTime'] = df_Raw.ReceivedSvTimeNanos  - df_Raw.ReceivedSvTimeUncertaintyNanos
print(df_Raw['TransmitTime'].describe())
df_Raw['TransmitTime'].hist(bins=2)
plt.show()

In [ ]:
# Full Bias Nano secons is zero 
df_Raw.info()

In [ ]:
# Bias Uncertainity NOnno is too large
print(df_Raw.BiasUncertaintyNanos.describe([0.25,0.50,0.6,0.7,0.8,0.9]))
df_Raw.boxplot(column='BiasUncertaintyNanos')
plt.show()


In [ ]:
print(df_Raw['ArrivalTime'].describe([0.25,0.5,0.6,0.7,0.8,0.9]))
df_Raw.boxplot(column = ['ArrivalTime'])
plt.show()

In [ ]:
# ConstellationType                          
df_Raw.ConstellationType.value_counts(normalize=True)

In [ ]:
# TimeNanos is empty 
df_Raw[df_Raw.TimeNanos.isnull()]

In [ ]:
# Max value should be 65535 
df_Raw.State.value_counts()

In [ ]:
#11  ReceivedSvTimeNanos                        57497 non-null  int64  
 #12  ReceivedSvTimeUncertaintyNanos             57497 non-null  int64  

df_Raw['TransmitTime'].describe()

In [ ]:
#AccumulatedDateRange - Max value allocated should be 31 as range of bits is only till 5 
df_Raw.AccumulatedDeltaRangeState.value_counts()

In [ ]:
# AccumulatedDeltaRangeUncertaintyMeters     is too high should range between 0 and 1 
df_Raw['AccumulatedDeltaRangeUncertaintyMeters'].describe()

In [ ]:
df_Raw.head()